# Philosophy Hunt

As much as they won't admit it, Philosophers love bragging rights. And what better bragging rights than of being the most mentioned philosopher in academia, the philosopher's equivalent of the trending.

This notebook does the following:
    
1. Generates a list of all philosophers
2. Scrapes the Stanford Encyclopedia of Philosophy for all entries
3. Finds the number of times a philosopher is mentioned across all entries
4. Ranks philosophers by order of mention

## Import Modules

In [75]:
from selenium import webdriver # allow launching browser
from selenium.webdriver.common.by import By # allow search with parameters
from selenium.webdriver.support.ui import WebDriverWait # allow waiting for page to load
from selenium.webdriver.support import expected_conditions as EC # determine whether the web page has loaded
from selenium.common.exceptions import TimeoutException # handling timeout situation
import pandas as pd
import numpy as np
import pickle
from collections import Counter

## Functions and Web Driver

In [3]:
driver_option = webdriver.ChromeOptions()
driver_option.add_argument(" — incognito")
chromedriver_path = '/Users/EyreCraggs/Documents/Coding/chromedriver' # Change this to your own chromedriver path!
def create_webdriver():
    return webdriver.Chrome(executable_path=chromedriver_path, chrome_options=driver_option)

In [4]:
#Edited version that also collects publication data

def scrape_url(url):
    new_browser = create_webdriver()
    new_browser.get(url)
 
 # Extract required data here
    
    #Para data
    allPara = new_browser.find_elements_by_xpath("//p")
    paraList = []
    for p in allPara:
        paraList.append(p.text)

    allText = "".join(paraList)
    
    #Publication data
    publication = new_browser.find_elements_by_xpath("//div[@id='pubinfo']/em")
    pubinfo = publication[0].text
    end = pubinfo.find(";")
    pubinfo = pubinfo[16:end]
 
    new_browser.quit()
 
    return pubinfo, allText

In [ ]:
def textReplace(repCharList,text):
    for char in repCharList:
        if char in text:
            text = text.replace(char," ")
    return text

## Scrape Philosopher Names

In [6]:
#list of philosopher name urls
wikiLinks = ["https://en.wikipedia.org/wiki/List_of_philosophers_(A%E2%80%93C)",
            "https://en.wikipedia.org/wiki/List_of_philosophers_(D%E2%80%93H)",
            "https://en.wikipedia.org/wiki/List_of_philosophers_(I%E2%80%93Q)",
            "https://en.wikipedia.org/wiki/List_of_philosophers_(R%E2%80%93Z)"]

In [7]:
philosopherNames = [] #the list we wish to populate

for link in wikiLinks:
    new_browser = create_webdriver()
    new_browser.get(link)
    allLinks = new_browser.find_elements_by_xpath("//ul/li")
    philLinks = allLinks[5:-50]
    for link in philLinks:
        philosopherNames.append(link.text)
    
    new_browser.quit()
    

/Users/EyreCraggs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [8]:
philosopherNames

['Adi Shankara, circa. 7th Century',
 'Nicola Abbagnano, (1901–1990)[2]',
 'Muhammad Abduh, (1849–1905)[4]',
 'Peter Abelard, (1079–1142)[1][2][3][4][5]',
 'Abhinavagupta (fl. c. 975 - 1025)[4]',
 'Isaac ben Judah Abravanel, (1437–1508)[1][4][5]',
 'Judah ben Isaac Abravanel, (1460?-1535?)[1][4]',
 'Marilyn McCord Adams, (born 1943)[3]',
 'Robert Adams, (born 1937)[3]',
 'Joseph Addison, (1672–1719)[2]',
 'Adelard of Bath, (12th century)[1]',
 'Alfred Adler, (1870–1937)[2]',
 'Theodor Adorno, (1903–1969)[1][2][3][4][5]',
 'Aenesidemus, (1st century BC)[2][4]',
 'Jamal al-Din al-Afghani, (1839–1897)[2][4]',
 'Rodolphus Agricola, (1443–1485)[4]',
 'Agrippa the Sceptic, (1st/2nd century)[2][4]',
 'Heinrich Cornelius Agrippa, (1436–1535)[2][4][5]',
 "Pierre d'Ailly, (1350–1420)[1][2][4]",
 'Kazimierz Ajdukiewicz, (1890–1963)[3][4]',
 'Hans Albert, (born 1921)',
 'Albert of Saxony (c. 1316-1390)[1][2][4][5]',
 'Albertus Magnus (or Saint Albert the Great or Albert of Cologne), (1193?-1280)[1

We want to extract just the names. That is, we wish to remove reference to the philosopher's lifespan.

In [9]:
cleanNames = []
bin1 = []
bin2 = []
for name in philosopherNames:
    try:
        bin1.append(name[:name.rindex(",")])
    except ValueError:
        bin1.append(name)
    except error as err:
        print(err)
        
for name in bin1:
    try:
        bin2.append(name[:name.index("(")-1])
    except ValueError:
        bin2.append(name)
    except error as err:
        print(err)
        
for name in bin2:
    try:
        cleanNames.append(name[:name.index("[")])
    except ValueError:
        cleanNames.append(name)
    except error as err:
        print(err)

In [10]:
cleanNames

['Adi Shankara',
 'Nicola Abbagnano',
 'Muhammad Abduh',
 'Peter Abelard',
 'Abhinavagupta',
 'Isaac ben Judah Abravanel',
 'Judah ben Isaac Abravanel',
 'Marilyn McCord Adams',
 'Robert Adams',
 'Joseph Addison',
 'Adelard of Bath',
 'Alfred Adler',
 'Theodor Adorno',
 'Aenesidemus',
 'Jamal al-Din al-Afghani',
 'Rodolphus Agricola',
 'Agrippa the Sceptic',
 'Heinrich Cornelius Agrippa',
 "Pierre d'Ailly",
 'Kazimierz Ajdukiewicz',
 'Hans Albert',
 'Albert of Saxony',
 'Albertus Magnus',
 'Joseph Albo',
 'Alcinous',
 'Alcmaeon of Croton',
 'Virgil Aldrich',
 'Yohanan ben Isaac Alemanno',
 'Alexander of Aphrodisias',
 'Alexander of Hales',
 'Samuel Alexander',
 'Dante Alighieri',
 'Antonio Aliotta',
 'Archibald Alison',
 'Abd al-Jabiri',
 'Abdullah al-Qasemi',
 'William Alston',
 'Johannes Althusius',
 'Louis Althusser',
 'Robert Alyngton',
 'Bhimrao Ambedkar',
 'Ambrose',
 "Abu'l Hasan Muhammad Ibn Yusuf al-'Amiri",
 'Ammonius Hermiae',
 'Ammonius Saccas',
 'André-Marie Ampère',
 'Ant

## Scrape Stanford Encyclopedia of Philosophy Text

We now want to find out whose name appears the most times in the SEP
A problem is that we have the philosopher's full name, say "Martin Heidegger" or "Thomas Henry Huxley", whereas often philosophers will be referenced using just their last name, less the ancient philosophers, at least in the Western tradition.



In [11]:
cleanNames[1].split(" ")[-1]

'Abbagnano'

We want to collect all the text in the SEP to count the number of times each philosopher is mentioned

First we collect the urls for all SEP pages

In [12]:
# Open the website
browser = create_webdriver()
browser.get("https://plato.stanford.edu/contents.html")

/Users/EyreCraggs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [13]:
# Extract all projects
projects = browser.find_elements_by_xpath("//div[@id='content']//ul/li/a")

In [14]:
# Extract information for each project
urlList = []
for proj in projects:
    urlList.append(proj.get_attribute('href'))

In [15]:
browser.quit()

In [285]:
urlList[:5]

['https://plato.stanford.edu/entries/abduction/',
 'https://plato.stanford.edu/entries/abelard/',
 'https://plato.stanford.edu/entries/abhidharma/',
 'https://plato.stanford.edu/entries/abilities/',
 'https://plato.stanford.edu/entries/abner-burgos/']

Extract the text and publication info from each SEP page and save it in a list.

In [20]:
pub_text = []
errorList = []

for i,link in enumerate(urlList):
    try:
        pub_text.append((scrape_url(link)))
        print(i)
    except:
        errorList.append(i)
    
    

/Users/EyreCraggs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


0
1
2
3
4
5
6
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


In [21]:
errorList

[7, 617]

In [258]:
#get data from the missing links, if possible
missingLinks = []
for i in errorList:
    missingLinks.append(urlList[i])
    

In [260]:
for i,link in enumerate(missingLinks):
    try:
        pub_text.append((scrape_url(link)))
        print(i)
    except:
        errorList.append(i)
    
    

/Users/EyreCraggs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


0


## Pickle Text

In [34]:
# save text list using pickle

with open('philPubText','wb') as handle:
    pickle.dump(pub_text,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open('philPubText','rb') as handle:
    b = pickle.load(handle)
    
print(pub_text == b)

True


## Most Cited Philosophers

We want to generate a string of all the philosopher text, without the publication data.


In [41]:
philNames = np.array(cleanNames)
philNames

array(['Adi Shankara', 'Nicola Abbagnano', 'Muhammad Abduh', ...,
       'Zhu Xi', 'Zhuang Zi', 'Slavoj Žižek'], dtype='<U46')

In [49]:
#generate a dataframe with full name and last name
nameCount = pd.DataFrame(philNames,columns = ["PhilNames"])

In [50]:
nameCount.head()

,PhilNames
0,Adi Shankara
1,Nicola Abbagnano
2,Muhammad Abduh
3,Peter Abelard
4,Abhinavagupta


In [284]:
#strip trailing whitespace from PhilNames
nameCount["PhilNames"] = [name.rstrip(" ") for name in nameCount["PhilNames"]]

#add last names
lastName = []
lastNameErrors = []
for name in nameCount["PhilNames"]:
    try:
        lastName.append(name[name.rindex(" ")+1:])
    except ValueError:
        lastName.append(name)
        lastNameErrors.append(name)
        
nameCount["LastName"] = np.array(lastName)

In [63]:
nameCount.head()

,PhilNames,LastName
0,Adi Shankara,Shankara
1,Nicola Abbagnano,Abbagnano
2,Muhammad Abduh,Abduh
3,Peter Abelard,Abelard
4,Abhinavagupta,Abhinavagupta


In [ ]:
#generate string file
philTextList = [pair[1] for pair in pub_text]
philText = ",".join(philTextList)

In [229]:
# generate list of punctuation to remove from text before analysis
import string
puncList = string.punctuation
puncList = puncList.replace("'","")

In [230]:
puncList = list(puncList)
puncList.append("'s")
puncList.append("—")
puncList.append("’s")
puncList.append("“")
puncList.append("”")

In [231]:
cleanPhilText = textReplace(puncList,philText.lower()) #clean philText

In [232]:
philCounter = Counter(cleanPhilText.split()) # generate a Counter object for all words in the SEP text

In [233]:
philCounter

Counter({'in': 524879,
         'the': 1442000,
         'philosophical': 16831,
         'literature': 4719,
         'term': 10334,
         'abduction': 276,
         'is': 516797,
         'used': 11512,
         'two': 34927,
         'related': 6901,
         'but': 97546,
         'different': 28730,
         'senses': 2749,
         'both': 26477,
         'refers': 2789,
         'to': 648621,
         'some': 60905,
         'form': 21058,
         'of': 1089879,
         'explanatory': 2998,
         'reasoning': 8252,
         'however': 28573,
         'historically': 1126,
         'first': 34434,
         'sense': 25673,
         'it': 208015,
         'place': 8840,
         'generating': 584,
         'hypotheses': 2504,
         'while': 18636,
         'which': 111051,
         'most': 26881,
         'frequently': 1444,
         'modern': 6681,
         'justifying': 631,
         'latter': 7114,
         'also': 48313,
         'often': 12803,
         'called': 11

In [240]:
# new column of the number of times the author's name appears
nameCount['LastNameCount'] = [philCounter[name.lower()] for name in nameCount["LastName"]] 

In [242]:
nameCount["FullNameCount"] = [philCounter[name.lower()] for name in nameCount["PhilNames"]]

In [243]:
nameCount.head()

,PhilNames,LastName,LastNameCount,FullNameCount
0,Adi Shankara,Shankara,5,0
1,Nicola Abbagnano,Abbagnano,0,0
2,Muhammad Abduh,Abduh,0,0
3,Peter Abelard,Abelard,635,0
4,Abhinavagupta,Abhinavagupta,19,19


In [257]:
nameCount.nlargest(40,'LastNameCount')

,PhilNames,LastName,LastNameCount,FullNameCount
1030,Henry More,More,52216,0
1031,Thomas More,More,52216,0
574,Pope Gregory I,I,40944,0
773,Justinian I,I,40944,0
1603,Yi I,I,40944,0
864,William Law,Law,14905,0
72,Aristotle,Aristotle,12194,12194
778,Immanuel Kant,Kant,9501,0
697,David Hume,Hume,6134,0
366,Paul de Man,Man,5868,0


## A First Hash at the Big Wigs

1. Aristotle, 12,194
2. Kant, 9,501
3. Hume, 6,134
4. Plato, 5,865
5. Russell, 4,452
6. Socrates, 4,277
7. John Locke, 4243
8. Rene Descartes, 4,037
9. Alan Turing, 2,905
10. Gottlob Frege, 2725

## Limitations of Analysis

In [ ]:
filter1 = nameCount["PhilNames"].str.contains("'")
nameCount[filter1]

- The analysis cannot distinguish between references to different people who share the same surname, e.g. John Stuart Mill and Harriet Taylor Mill.
- The analysis does not accurately parse the last names of Chinese philosophers, whose last names come first.
- The analysis does not accurately parse the last names of certain French (and perhaps Spanish) philosophers whose last names can be two words, e.g. "du Man"
- The analysis does not distinguish between occurances of a last name from common words where the two are identical, for instance distinguishing "Law" in "William Law" and "The rule of Law".